In [ ]:
!pip install transformers pandas scikit-learn


In [ ]:
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report
import torch
from google.colab import files


In [ ]:
# Load the sentiment analysis pipeline and model
sentiment_pipeline = pipeline("sentiment-analysis")
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [5]:
# Step 4.1: Upload the dataset
uploaded = files.upload()

# Load and preprocess the dataset (assuming columns 'Text' and 'Sentiment')
file_name = list(uploaded.keys())[0]
data = pd.read_csv(file_name)
data = data[['Text', 'Sentiment']].dropna()

# Step 4.2: Map ground truth sentiments to match model output format
# (e.g., "POSITIVE", "NEGATIVE")
data['Sentiment'] = data['Sentiment'].str.upper()

# Step 4.3: Apply sentiment analysis to each text entry and get predictions
data['Predicted_Sentiment'] = data['Text'].apply(lambda x: sentiment_pipeline(x)[0]['label'])

# Step 4.4: Calculate Accuracy
accuracy = accuracy_score(data['Sentiment'], data['Predicted_Sentiment'])
print(f"Model Accuracy: {accuracy:.2f}")

# Step 4.5: Detailed Classification Report
print("Classification Report:")
print(classification_report(data['Sentiment'], data['Predicted_Sentiment']))


Saving sentimentdataset.csv to sentimentdataset.csv
Model Accuracy: 0.00
Classification Report:
                        precision    recall  f1-score   support

         ACCEPTANCE          0.00      0.00      0.00       3.0
      ACCEPTANCE             0.00      0.00      0.00       5.0
       ACCOMPLISHMENT        0.00      0.00      0.00       3.0
           ADMIRATION        0.00      0.00      0.00       1.0
         ADMIRATION          0.00      0.00      0.00       2.0
        ADMIRATION           0.00      0.00      0.00       1.0
         ADORATION           0.00      0.00      0.00       2.0
       ADRENALINE            0.00      0.00      0.00       1.0
            ADVENTURE        0.00      0.00      0.00       3.0
         AFFECTION           0.00      0.00      0.00       2.0
            AMAZEMENT        0.00      0.00      0.00       1.0
          AMBIVALENCE        0.00      0.00      0.00       1.0
      AMBIVALENCE            0.00      0.00      0.00       5.0
       

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_

In [6]:
def explain_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    inputs_embeds = model.get_input_embeddings()(inputs['input_ids'])
    inputs_embeds.retain_grad()

    outputs = model(inputs_embeds=inputs_embeds)
    scores = outputs.logits
    predicted_class = scores.argmax().item()
    sentiment = "POSITIVE" if predicted_class == 1 else "NEGATIVE"

    model.zero_grad()
    scores[0, predicted_class].backward()

    token_importance = inputs_embeds.grad.abs().sum(dim=-1).squeeze()
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze())
    explanation = [(tokens[i], round(token_importance[i].item(), 4)) for i in range(1, len(tokens) - 1)]
    explanation = sorted(explanation, key=lambda x: x[1], reverse=True)

    print(f"Sentiment: {sentiment}")
    print("Explanation (Top contributing tokens):")
    for token, importance in explanation[:5]:
        print(f"Token: {token}, Importance: {importance}")


In [ ]:
print("Sentiment Analysis with Explanation - Live Input")
print("Type 'exit' to quit the loop.")

def explain_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    inputs_embeds = model.get_input_embeddings()(inputs['input_ids'])
    inputs_embeds.retain_grad()

    outputs = model(inputs_embeds=inputs_embeds)
    scores = outputs.logits
    predicted_class = scores.argmax().item()
    sentiment = "POSITIVE" if predicted_class == 1 else "NEGATIVE"

    model.zero_grad()
    scores[0, predicted_class].backward()

    token_importance = inputs_embeds.grad.abs().sum(dim=-1).squeeze()
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze())
    explanation = [tokens[i] for i in range(1, len(tokens) - 1)]

    # Display sentiment and most important tokens without importance values
    print(f"Sentiment: {sentiment}")
    print("Explanation (Top contributing tokens):")
    print(" ".join(explanation[:5]))  # Show top 5 tokens for brevity

# Start a continuous loop for live input
while True:
    # Get user input
    user_input = input("Enter text for sentiment analysis: ")

    # Break the loop if the user types 'exit'
    if user_input.lower() == "exit":
        print("Exiting sentiment analysis loop.")
        break

    # Run sentiment analysis and explanation
    sentiment_result = sentiment_pipeline(user_input)
    predicted_sentiment = sentiment_result[0]['label']
    confidence_score = sentiment_result[0]['score']

    print(f"Predicted Sentiment: {predicted_sentiment} (Confidence: {confidence_score:.2f})")
    print("Explanation for sentiment prediction:")
    explain_sentiment(user_input)
    print("-" * 50)


Sentiment Analysis with Explanation - Live Input
Type 'exit' to quit the loop.
Enter text for sentiment analysis: I'm okay because my dad purchased  new phone
Predicted Sentiment: POSITIVE (Confidence: 1.00)
Explanation for sentiment prediction:
Sentiment: POSITIVE
Explanation (Top contributing tokens):
i ' m okay because
--------------------------------------------------
Enter text for sentiment analysis: I'm okay because my dad purchased  new phone because i spent my money currently i don't have any money
Predicted Sentiment: POSITIVE (Confidence: 0.98)
Explanation for sentiment prediction:
Sentiment: POSITIVE
Explanation (Top contributing tokens):
i ' m okay because
--------------------------------------------------
